In [ ]:
from tkinter import Tk, Button, Label, filedialog, Scale, Listbox, messagebox
from pygame import mixer
from mutagen.mp3 import MP3
import time
import random

class MusicPlayer:
    def __init__(self, window):
        window.geometry('400x400')
        window.title('Iris Player')
        window.resizable(0, 0)
        window.configure(bg="lightgray")

        self.music_files = []
        self.current_track_index = -1
        self.shuffle_mode = False
        self.playing_state = False

        self.label_track = Label(window, text="No track selected", font=("Arial", 12), bg="lightgray")
        self.label_track.pack(pady=10)

        button_load = Button(window, text="Load", width=10, font=("Arial", 10), command=self.load, bg="lightblue", fg="black")
        button_load.pack()

        button_play = Button(window, text="Play", width=10, font=("Arial", 10), command=self.play, bg="lightgreen", fg="black")
        button_play.pack()

        button_pause = Button(window, text="Pause", width=10, font=("Arial", 10), command=self.pause, bg="yellow", fg="black")
        button_pause.pack()

        button_stop = Button(window, text="Stop", width=10, font=("Arial", 10), command=self.stop, bg="red", fg="black")
        button_stop.pack()

        self.volume_scale = Scale(window, from_=0, to=100, orient="horizontal", length=200, label="Volume", font=("Arial", 10), command=self.set_volume, bg="lightgray")
        self.volume_scale.set(50)
        self.volume_scale.pack(pady=10)

        self.label_duration = Label(window, text="Duration: 00:00", font=("Arial", 10), bg="lightgray")
        self.label_duration.pack()

        self.progress_bar = Scale(window, from_=0, to=100, orient="horizontal", length=200, showvalue=False, font=("Arial", 10), bg="white", troughcolor="lightgray", sliderlength=10, sliderrelief="raised")
        self.progress_bar.set(0)
        self.progress_bar.pack(pady=10)

        self.playlist_label = Label(window, text="Playlist", font=("Arial", 12), bg="lightgray")
        self.playlist_label.pack()

        self.playlist = Listbox(window, selectbackground="gray", selectforeground="white", font=("Arial", 10), width=40)
        self.playlist.pack(pady=5)

        button_previous = Button(window, text="Previous", width=10, font=("Arial", 10), command=self.previous_track, bg="lightgray", fg="black")
        button_previous.pack()

        button_next = Button(window, text="Next", width=10, font=("Arial", 10), command=self.next_track, bg="lightgray", fg="black")
        button_next.pack()

        button_add = Button(window, text="Add to Playlist", width=15, font=("Arial", 10), command=self.add_to_playlist, bg="lightgray", fg="black")
        button_add.pack(pady=5)

        button_delete = Button(window, text="Delete from Playlist", width=15, font=("Arial", 10), command=self.delete_from_playlist, bg="lightgray", fg="black")
        button_delete.pack()

        self.load_playlist()

    def load_playlist(self):
        try:
            with open("playlist.txt", "r") as file:
                self.music_files = file.read().splitlines()
                for file in self.music_files:
                    self.playlist.insert("end", file)
        except FileNotFoundError:
            pass

    def save_playlist(self):
        with open("playlist.txt", "w") as file:
            file.write("\n".join(self.music_files))

    def load(self):
        music_files = filedialog.askopenfilenames(filetypes=[("MP3 Files", "*.mp3"), ("All Files", "*.*")])
        if music_files:
            new_files = list(music_files)
            self.music_files += new_files
            for file in new_files:
                self.playlist.insert("end", file)
            self.save_playlist()
            if self.current_track_index == -1:
                self.current_track_index = 0
            self.label_track["text"] = f"Track: {self.music_files[self.current_track_index]}"
            self.progress_bar["value"] = 0
            self.update_duration_label()

    def play(self):
        if self.music_files:
            if not self.playing_state:
                track = self.music_files[self.current_track_index]
                mixer.init()
                mixer.music.load(track)
                mixer.music.play()
                self.label_track["text"] = f"Track: {track}"
                self.playing_state = True

    def pause(self):
        if self.playing_state:
            mixer.music.pause()
            self.playing_state = False
        else:
            mixer.music.unpause()
            self.playing_state = True

    def stop(self):
        if self.playing_state:
            mixer.music.stop()
            self.playing_state = False

    def set_volume(self, volume):
        if self.music_files:
            volume = int(volume) / 100
            mixer.music.set_volume(volume)

    def update_duration_label(self):
        if self.music_files:
            if self.current_track_index >= 0 and self.current_track_index < len(self.music_files):
                track = self.music_files[self.current_track_index]
                audio = MP3(track)
                duration = int(audio.info.length)
                minutes = duration // 60
                seconds = duration % 60
                duration_str = f"Duration: {minutes:02d}:{seconds:02d}"
                self.label_duration["text"] = duration_str
                self.progress_bar["to"] = duration

    def update_progress_bar(self):
        if self.music_files and self.current_track_index >= 0 and self.current_track_index < len(self.music_files):
            track = self.music_files[self.current_track_index]
            audio = MP3(track)
            duration = int(audio.info.length)
            if duration > 0 and self.playing_state:
                progress = mixer.music.get_pos() // 1000
                self.progress_bar.set(progress)
                if progress >= duration:
                    self.progress_bar.set(0)
                    self.playing_state = False
                    self.next_track()

        self.progress_bar.after(1000, self.update_progress_bar)

    def previous_track(self):
        if self.music_files:
            if self.shuffle_mode:
                self.shuffle_tracks()
            else:
                self.current_track_index -= 1
                if self.current_track_index < 0:
                    self.current_track_index = len(self.music_files) - 1
            self.stop()
            self.play()

    def next_track(self):
        if self.music_files:
            if self.shuffle_mode:
                self.shuffle_tracks()
            else:
                self.current_track_index += 1
                if self.current_track_index >= len(self.music_files):
                    self.current_track_index = 0
            self.stop()
            self.play()

    def shuffle_tracks(self):
        if self.music_files:
            self.stop()
            random.shuffle(self.music_files)
            self.current_track_index = 0
            self.play()

    def add_to_playlist(self):
        music_files = filedialog.askopenfilenames(filetypes=[("MP3 Files", "*.mp3"), ("All Files", "*.*")])
        if music_files:
            new_files = list(music_files)
            self.music_files += new_files
            for file in new_files:
                self.playlist.insert("end", file)
            self.save_playlist()

    def delete_from_playlist(self):
        if self.playlist.curselection():
            selected_index = self.playlist.curselection()[0]
            selected_track = self.music_files[selected_index]
            confirm = messagebox.askyesno("Delete Confirmation", f"Are you sure you want to delete '{selected_track}' from the playlist?")
            if confirm:
                self.music_files.pop(selected_index)
                self.playlist.delete(selected_index)
                self.save_playlist()

    def select_track(self, event):
        if self.playlist.curselection():
            self.current_track_index = self.playlist.curselection()[0]
            self.stop()
            self.play()

root = Tk()
root.configure(bg="lightgray")
app = MusicPlayer(root)
app.update_progress_bar()
root.mainloop()
